In [12]:
import tensorflow as tf
import numpy as np
import os
from numpy import genfromtxt
from keras import backend as K
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.core import Lambda, Flatten, Dense
 

In [24]:

def conv2d_bn(x,
              layer=None,
              cv1_out=None,
              cv1_filter=(1, 1),
              cv1_strides=(1, 1),
              cv2_out=None,
              cv2_filter=(3, 3),
              cv2_strides=(1, 1),
              padding=None):
    num = '' if cv2_out == None else '1'
    tensor = Conv2D(cv1_out, cv1_filter, strides=cv1_strides, data_format='channels_last', name=layer+'_conv'+num)(x)
    tensor = BatchNormalization(axis=1, epsilon=0.00001, name=layer+'_bn'+num)(tensor)
    tensor = Activation('relu')(tensor)
    if padding == None:
        return tensor
    tensor = ZeroPadding2D(padding=padding, data_format='channels_last')(tensor)
    if cv2_out == None:
        return tensor
    tensor = Conv2D(cv2_out, cv2_filter, strides=cv2_strides, data_format='channels_last', name=layer+'_conv'+'2')(tensor)
    tensor = BatchNormalization(axis=1, epsilon=0.00001, name=layer+'_bn'+'2')(tensor)
    tensor = Activation('relu')(tensor)
    return tensor

In [25]:
def inception_block_1a(X):
    """
    Implementation of an inception block
    """
    X_1x1 = Conv2D(64, (1, 1), data_format='channels_last', name='inception_3a_1x1_conv')(X)
    X_1x1 = BatchNormalization(axis=-1, epsilon=0.00001, name='inception_3a_1x1_bn')(X_1x1)
    X_1x1 = Activation('relu')(X_1x1)
    print('Chain 1: ', X_1x1.shape)
    
    X_3x3 = Conv2D(96, (1, 1), data_format='channels_last', name ='inception_3a_3x3_conv1')(X)
    X_3x3 = BatchNormalization(axis=-1, epsilon=0.00001, name = 'inception_3a_3x3_bn1')(X_3x3)
    X_3x3 = Activation('relu')(X_3x3)
    X_3x3 = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(X_3x3)
    X_3x3 = Conv2D(128, (3, 3), data_format='channels_last', name='inception_3a_3x3_conv2')(X_3x3)
    X_3x3 = BatchNormalization(axis=-1, epsilon=0.00001, name='inception_3a_3x3_bn2')(X_3x3)
    X_3x3 = Activation('relu')(X_3x3)
    print('Chain 2: ', X_3x3.shape)
    
    X_5x5 = Conv2D(16, (1, 1), data_format='channels_last', name='inception_3a_5x5_conv1')(X)
    X_5x5 = BatchNormalization(axis=-1, epsilon=0.00001, name='inception_3a_5x5_bn1')(X_5x5)
    X_5x5 = Activation('relu')(X_5x5)
    X_5x5 = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(X_5x5)
    X_5x5 = Conv2D(32, (5, 5), data_format='channels_last', name='inception_3a_5x5_conv2')(X_5x5)
    X_5x5 = BatchNormalization(axis=-1, epsilon=0.00001, name='inception_3a_5x5_bn2')(X_5x5)
    X_5x5 = Activation('relu')(X_5x5)
    print('Chain 3: ', X_5x5.shape)
    
    X_pool = MaxPooling2D(pool_size=3, strides=2, data_format='channels_last')(X)
    X_pool = Conv2D(32, (1, 1), data_format='channels_last', name='inception_3a_pool_conv')(X_pool)
    X_pool = BatchNormalization(axis=-1, epsilon=0.00001, name='inception_3a_pool_bn')(X_pool)
    X_pool = Activation('relu')(X_pool)
    X_pool = ZeroPadding2D(padding=((3, 4), (3, 4)), data_format='channels_last')(X_pool)
    print('Chain 4: ', X_pool.shape)

    # CONCAT
    inception = concatenate([X_3x3, X_5x5, X_pool, X_1x1], axis=-1)
    print('inception3a : ', inception.shape)
    return inception

def inception_block_1b(X):
    X_3x3 = Conv2D(96, (1, 1), data_format='channels_last', name='inception_3b_3x3_conv1')(X)
    X_3x3 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_3x3_bn1')(X_3x3)
    X_3x3 = Activation('relu')(X_3x3)
    X_3x3 = ZeroPadding2D(padding=(1, 1), data_format='channels_last')(X_3x3)
    X_3x3 = Conv2D(128, (3, 3), data_format='channels_last', name='inception_3b_3x3_conv2')(X_3x3)
    X_3x3 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_3x3_bn2')(X_3x3)
    X_3x3 = Activation('relu')(X_3x3)
    print('Chain 2: ', X_3x3.shape)

    X_5x5 = Conv2D(32, (1, 1), data_format='channels_last', name='inception_3b_5x5_conv1')(X)
    X_5x5 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_5x5_bn1')(X_5x5)
    X_5x5 = Activation('relu')(X_5x5)
    X_5x5 = ZeroPadding2D(padding=(2, 2), data_format='channels_last')(X_5x5)
    X_5x5 = Conv2D(64, (5, 5), data_format='channels_last', name='inception_3b_5x5_conv2')(X_5x5)
    X_5x5 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_5x5_bn2')(X_5x5)
    X_5x5 = Activation('relu')(X_5x5)
    print('Chain 3: ', X_5x5.shape)

    X_pool = AveragePooling2D(pool_size=(3, 3), strides=(3, 3), data_format='channels_last')(X)
    X_pool = Conv2D(64, (1, 1), data_format='channels_last', name='inception_3b_pool_conv')(X_pool)
    X_pool = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_pool_bn')(X_pool)
    X_pool = Activation('relu')(X_pool)
    X_pool = ZeroPadding2D(padding=(4, 4), data_format='channels_last')(X_pool)
    print('Chain 4: ', X_pool.shape)

    X_1x1 = Conv2D(64, (1, 1), data_format='channels_last', name='inception_3b_1x1_conv')(X)
    X_1x1 = BatchNormalization(axis=1, epsilon=0.00001, name='inception_3b_1x1_bn')(X_1x1)
    X_1x1 = Activation('relu')(X_1x1)
    print('Chain 1: ', X_1x1.shape)

    inception = concatenate([X_3x3, X_5x5, X_pool, X_1x1], axis=-1)

    return inception


def inception_block_1c(X):
    X_3x3 = conv2d_bn(X,
                           layer='inception_3c_3x3',
                           cv1_out=128,
                           cv1_filter=(1, 1),
                           cv2_out=256,
                           cv2_filter=(3, 3),
                           cv2_strides=(2, 2),
                           padding=(1, 1))
    print('Chain 2: ', X_3x3.shape)
    X_5x5 = conv2d_bn(X,
                           layer='inception_3c_5x5',
                           cv1_out=32,
                           cv1_filter=(1, 1),
                           cv2_out=64,
                           cv2_filter=(5, 5),
                           cv2_strides=(2, 2),
                           padding=(2, 2))
    print('Chain 3: ', X_5x5.shape)

    X_pool = MaxPooling2D(pool_size=3, strides=2, data_format='channels_first')(X)
    X_pool = ZeroPadding2D(padding=((0, 1), (0, 1)), data_format='channels_first')(X_pool)
    print('Chain 3: ', X_pool.shape)
    inception = concatenate([X_3x3, X_5x5, X_pool], axis=-1)

    return inception

def faceRecoModel(input_shape):
    """
    Implementation of the Inception model used for FaceNet

    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)
    print('X_input ', X_input)
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    print('ZeroPadding2D ', X)
    # First Block
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(X)
    X = BatchNormalization(axis=1, name='bn1')(X)
    X = Activation('relu')(X)
    print('Activation ', X)
    # Zero-Padding + MAXPOOL
    X = ZeroPadding2D((1, 1))(X)
    print('ZeroPadding2D ', X)
    X = MaxPooling2D((3, 3), strides=2)(X)
    print('MaxPooling2D ', X)
    # Second Block
    X = Conv2D(64, (1, 1), strides=(1, 1), name='conv2')(X)
    X = BatchNormalization(axis=1, epsilon=0.00001, name='bn2')(X)
    X = Activation('relu')(X)
    
    # Zero-Padding + MAXPOOL
    X = ZeroPadding2D((1, 1))(X)
    
    # Second Block
    X = Conv2D(192, (3, 3), strides=(1, 1), name='conv3')(X)
    X = BatchNormalization(axis=1, epsilon=0.00001, name='bn3')(X)
    X = Activation('relu')(X)
    
    # Zero-Padding + MAXPOOL
    X = ZeroPadding2D((1, 1))(X)
    X = MaxPooling2D(pool_size=3, strides=2)(X)
    print (X.shape)
    # Inception 1: a/b/c

    X = inception_block_1a(X)
    X = inception_block_1b(X)
    X = inception_block_1c(X)
    
    model = Model(inputs = X_input, outputs = X, name='FaceRecoModel')
    return model

FRmodel = faceRecoModel(input_shape=(96, 96, 3))

# get only the conv weight


X_input  Tensor("input_14:0", shape=(?, 96, 96, 3), dtype=float32)
ZeroPadding2D  Tensor("zero_padding2d_128/Pad:0", shape=(?, 102, 102, 3), dtype=float32)
Activation  Tensor("activation_184/Relu:0", shape=(?, 48, 48, 64), dtype=float32)
ZeroPadding2D  Tensor("zero_padding2d_129/Pad:0", shape=(?, 50, 50, 64), dtype=float32)
MaxPooling2D  Tensor("max_pooling2d_43/MaxPool:0", shape=(?, 24, 24, 64), dtype=float32)
(?, 12, 12, 192)
Chain 1:  (?, 12, 12, 64)
Chain 2:  (?, 12, 12, 128)
Chain 3:  (?, 12, 12, 32)
Chain 4:  (?, 12, 12, 32)
inception3a :  (?, 12, 12, 256)
Chain 2:  (?, 12, 12, 128)
Chain 3:  (?, 12, 12, 64)
Chain 4:  (?, 12, 12, 64)
Chain 1:  (?, 12, 12, 64)
Chain 2:  (?, 6, 6, 256)
Chain 3:  (?, 6, 6, 64)
Chain 3:  (?, 12, 6, 160)


ValueError: `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 6, 6, 256), (None, 6, 6, 64), (None, 12, 6, 160)]

In [ ]:
from collections import defaultdict
moduleWeightDict = defaultdict(lambda: defaultdict())
parentPath = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceNet-Inception/"
layer_name = ['conv1', 'conv2', 'conv3', 
              'inception_3a_1x1_conv', 
              'inception_3a_3x3_conv1', 
              'inception_3a_3x3_conv2',
              'inception_3a_5x5_conv1', 
              'inception_3a_5x5_conv2', 
              'inception_3a_pool_conv',
              'inception_3b_1x1_conv', 
              'inception_3b_3x3_conv1', 
              'inception_3b_3x3_conv2',
              'inception_3b_5x5_conv1', 
              'inception_3b_5x5_conv2', 
              'inception_3b_pool_conv',
              'inception_3c_3x3_conv1',
              'inception_3c_3x3_conv2',
              'inception_3c_5x5_conv1', 
              'inception_3c_5x5_conv2', ]#, 'conv1_w']

for name in layer_name:
    conv1_w = np.genfromtxt(parentPath+"/"+name+'_w.csv', delimiter=',', dtype=None)
    conv1_w = np.reshape(conv1_w, convShape[name])
    conv1_w = np.transpose(conv1_w, (2, 3, 1, 0))
    
    conv1_b = np.genfromtxt(parentPath+"/"+name+'_b.csv', delimiter=',', dtype=None)
    moduleWeightDict[name]['w'] = conv1_w
    moduleWeightDict[name]['b'] = conv1_b
    print (name)
    print (conv1_w.shape)
    print (conv1_b.shape)
# FRmodel.get_layer('conv1').set_weights([conv1_w, conv1_b])


# moduleWeightDict = {}
# parentPath = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceNet-Inception/"
# module_weight = 'conv1_w'
# module_bias = 'conv1_b'
# conv1_w = np.genfromtxt(parentPath+"/"+module_weight+'.csv', delimiter=',', dtype=None)
# conv1_w = np.reshape(conv1_w, [64, 3, 7, 7])
# conv1_w = np.transpose(conv1_w, (2, 3, 1, 0))

# conv1_b = np.genfromtxt(parentPath+"/"+module_bias+'.csv', delimiter=',', dtype=None)
# print (conv1_w.shape)
# print (conv1_b.shape)
# FRmodel.get_layer('conv1').set_weights([conv1_w, conv1_b])

In [3]:
FRmodel

In [5]:
picPath = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Models/FaceRecognition/camera_0.jpg'

from scipy import misc

def readImage(imagePath):
    '''
        The input data is is in the shape of [nh, nw, nc], convert it to [nc, nh, nw]
    '''
    image = misc.imread(picPath)
    
    img = np.around(np.transpose(image, (2,0,1))/255.0, decimals=12) #(2,0,1) = [nc, nh, nw]
    print (image.shape)

(96, 96, 3)


/Users/sam/App-Setup/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """


In [4]:
import cv2

ImportError: dlopen(/Users/sam/App-Setup/anaconda/lib/python3.6/site-packages/cv2.cpython-36m-darwin.so, 2): Library not loaded: @rpath/libintl.8.dylib
  Referenced from: /Users/sam/App-Setup/anaconda/lib/libharfbuzz.0.dylib
  Reason: image not found

In [6]:
image

array([[[247, 254, 213],
        [250, 255, 225],
        [252, 255, 223],
        ..., 
        [221, 224, 179],
        [244, 247, 190],
        [249, 255, 181]],

       [[251, 254, 223],
        [253, 253, 243],
        [254, 252, 240],
        ..., 
        [242, 247, 183],
        [235, 235, 199],
        [212, 215, 158]],

       [[254, 255, 221],
        [255, 253, 250],
        [255, 251, 255],
        ..., 
        [238, 246, 161],
        [237, 234, 201],
        [237, 235, 196]],

       ..., 
       [[253, 252, 255],
        [246, 245, 253],
        [217, 215, 226],
        ..., 
        [255, 255, 255],
        [253, 253, 253],
        [255, 255, 255]],

       [[213, 211, 222],
        [178, 176, 187],
        [155, 153, 166],
        ..., 
        [255, 255, 255],
        [253, 253, 255],
        [255, 255, 255]],

       [[156, 154, 165],
        [143, 141, 154],
        [145, 143, 157],
        ..., 
        [229, 228, 233],
        [254, 254, 255],
        [255, 255,